In [27]:
## import (and install if necessary) pandas
%pip install pandas
import pandas as pd


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
df = pd.read_csv("full_data.csv"
                 ,parse_dates=['TripDate','StartDateTime','EndDateTime','CustomerBirthDate']
                 )
## process cost monetary data
df['Cost'] = df['Cost'].str.replace(',', '')
df['Cost'] = df['Cost'].str.replace('$', '')
df['Cost'] = df['Cost'].str.replace('(', '')
df['Cost'] = df['Cost'].str.replace(')', '')
df['Cost'] = df['Cost'].astype(float)
print(df.info())
#print(df.head(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160755 entries, 0 to 1160754
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   CustomerID         1160755 non-null  int64         
 1   CustomerSex        1160755 non-null  object        
 2   CustomerBirthDate  1160417 non-null  datetime64[ns]
 3   TripDate           1160755 non-null  datetime64[ns]
 4   VehicleID          1160755 non-null  int64         
 5   RunActualID        1160755 non-null  int64         
 6   StartDateTime      1160755 non-null  datetime64[ns]
 7   EndDateTime        1160755 non-null  datetime64[ns]
 8   PickupX            1110755 non-null  float64       
 9   PickupY            1110755 non-null  float64       
 10  DropoffX           1109028 non-null  float64       
 11  DropoffY           1109028 non-null  float64       
 12  Cost               1160538 non-null  float64       
 13  CollectedFareType  1147941 

In [29]:
## summary counts
runActualID_counts = df.groupby(['RunActualID'])['RunActualID'].count()
max_runActualIDs = runActualID_counts.max().item()
print(f"Maximum Number of rows per RunActualID: {max_runActualIDs}")
runActualID_vehicleID = df.groupby('RunActualID')['VehicleID'].nunique()
runActualIDs_more_than_one_vehicleID = runActualID_vehicleID[runActualID_vehicleID != 1]
print(f"Any RunActualIDs that have more than one VehicleID: {runActualIDs_more_than_one_vehicleID.nunique()}")
print(f"Unique number of RunActualIDs: {df['RunActualID'].nunique()}")
print(f"Unique number of CustomerIDs: {df['CustomerID'].nunique()}")
print(f"Unique number of VehicleIDs: {df['VehicleID'].nunique()}")

Maximum Number of rows per RunActualID: 18
Any RunActualIDs that have more than one VehicleID: 0
Unique number of RunActualIDs: 213272
Unique number of CustomerIDs: 10215
Unique number of VehicleIDs: 538


In [30]:
dates_per_runactualIDs = df.groupby(['RunActualID']).agg({'EndDateTime': 'max', 'StartDateTime':'min'})
dates_per_runactualIDs['EndDate'] = pd.to_datetime(dates_per_runactualIDs['EndDateTime'].dt.date)
dates_per_runactualIDs['StartDate'] = pd.to_datetime(dates_per_runactualIDs['StartDateTime'].dt.date)
dates_per_runactualIDs['diff'] = (dates_per_runactualIDs['EndDateTime'] - dates_per_runactualIDs['StartDateTime']).dt.total_seconds()
seconds_in_day = 86400
runactualIDs_over_day = dates_per_runactualIDs[dates_per_runactualIDs['diff'] > seconds_in_day]
#print(runactualIDs_over_day['RunActualId'].nunique())

print(f"""Number of RunActualIDs lasting over 24 hours: {runactualIDs_over_day.index.nunique()}
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv""")
runactualIDs_over_day.to_csv("runactualIDs_over_day.csv", columns = ['StartDateTime','EndDateTime'])

Number of RunActualIDs lasting over 24 hours: 4
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv


In [ ]:
# generate field definitions
field_tuple = ()
for x in range(max_runActualIDs):
    y = x + 1
    field_tuple = field_tuple + (f"customerID_{y}=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"startdatetime_{y}=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"pickupx_{y}=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"pickupy_{y}=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"enddatetime_{y}=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"dropoffx_{y}=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"dropoffy_{y}=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"cost_{y}=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"collected_fare_type_{y}=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head({y}).tail(1))"
                                 )
print(field_tuple)

In [54]:
df_route_fields = df[['RunActualID','CustomerID','StartDateTime','PickupX','PickupY','EndDateTime','DropoffX','DropoffY','Cost','CollectedFareType']]
df_route_fields = df_route_fields.sort_values(by=['RunActualID','StartDateTime'])
df_route_fields = df_route_fields.reset_index()

# there is 100% a better way to do this, just not today:
df_routes = df_route_fields.groupby('RunActualID').agg(
    total_customers=pd.NamedAgg(column='CustomerID', aggfunc='count'),
    customerID_1=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(1).tail(1)),
    startdatetime_1=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(1).tail(1)),
    pickupx_1=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(1).tail(1)),
    pickupy_1=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(1).tail(1)),
    enddatetime_1=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(1).tail(1)),
    dropoffx_1=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(1).tail(1)),
    dropoffy_1=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(1).tail(1)),
    cost_1=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(1).tail(1)),
    collected_fare_type_1=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(1).tail(1)),
    customerID_2=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(2).tail(1)),
    startdatetime_2=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(2).tail(1)),
    pickupx_2=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(2).tail(1)),
    pickupy_2=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(2).tail(1)),
    enddatetime_2=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(2).tail(1)),
    dropoffx_2=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(2).tail(1)),
    dropoffy_2=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(2).tail(1)),
    cost_2=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(2).tail(1)),
    collected_fare_type_2=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(2).tail(1)),
    customerID_3=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(3).tail(1)),
    startdatetime_3=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(3).tail(1)),
    pickupx_3=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(3).tail(1)),
    pickupy_3=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(3).tail(1)),
    enddatetime_3=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(3).tail(1)),
    dropoffx_3=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(3).tail(1)),
    dropoffy_3=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(3).tail(1)),
    cost_3=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(3).tail(1)),
    collected_fare_type_3=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(3).tail(1)),
    customerID_4=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(4).tail(1)),
    startdatetime_4=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(4).tail(1)),
    pickupx_4=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(4).tail(1)),
    pickupy_4=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(4).tail(1)),
    enddatetime_4=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(4).tail(1)),
    dropoffx_4=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(4).tail(1)),
    dropoffy_4=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(4).tail(1)),
    cost_4=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(4).tail(1)),
    collected_fare_type_4=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(4).tail(1)),
    customerID_5=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(5).tail(1)),
    startdatetime_5=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(5).tail(1)),
    pickupx_5=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(5).tail(1)),
    pickupy_5=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(5).tail(1)),
    enddatetime_5=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(5).tail(1)),
    dropoffx_5=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(5).tail(1)),
    dropoffy_5=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(5).tail(1)),
    cost_5=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(5).tail(1)),
    collected_fare_type_5=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(5).tail(1)),
    customerID_6=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(6).tail(1)),
    startdatetime_6=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(6).tail(1)),
    pickupx_6=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(6).tail(1)),
    pickupy_6=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(6).tail(1)),
    enddatetime_6=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(6).tail(1)),
    dropoffx_6=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(6).tail(1)),
    dropoffy_6=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(6).tail(1)),
    cost_6=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(6).tail(1)),
    collected_fare_type_6=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(6).tail(1)),
    customerID_7=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(7).tail(1)),
    startdatetime_7=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(7).tail(1)),
    pickupx_7=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(7).tail(1)),
    pickupy_7=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(7).tail(1)),
    enddatetime_7=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(7).tail(1)),
    dropoffx_7=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(7).tail(1)),
    dropoffy_7=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(7).tail(1)),
    cost_7=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(7).tail(1)),
    collected_fare_type_7=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(7).tail(1)),
    customerID_8=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(8).tail(1)),
    startdatetime_8=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(8).tail(1)),
    pickupx_8=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(8).tail(1)),
    pickupy_8=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(8).tail(1)),
    enddatetime_8=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(8).tail(1)),
    dropoffx_8=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(8).tail(1)),
    dropoffy_8=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(8).tail(1)),
    cost_8=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(8).tail(1)),
    collected_fare_type_8=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(8).tail(1)),
    customerID_9=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(9).tail(1)),
    startdatetime_9=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(9).tail(1)),
    pickupx_9=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(9).tail(1)),
    pickupy_9=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(9).tail(1)),
    enddatetime_9=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(9).tail(1)),
    dropoffx_9=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(9).tail(1)),
    dropoffy_9=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(9).tail(1)),
    cost_9=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(9).tail(1)),
    collected_fare_type_9=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(9).tail(1)),
    customerID_10=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(10).tail(1)),
    startdatetime_10=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(10).tail(1)),
    pickupx_10=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(10).tail(1)),
    pickupy_10=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(10).tail(1)),
    enddatetime_10=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(10).tail(1)),
    dropoffx_10=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(10).tail(1)),
    dropoffy_10=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(10).tail(1)),
    cost_10=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(10).tail(1)),
    collected_fare_type_10=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(10).tail(1)),
    customerID_11=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(11).tail(1)),
    startdatetime_11=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(11).tail(1)),
    pickupx_11=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(11).tail(1)),
    pickupy_11=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(11).tail(1)),
    enddatetime_11=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(11).tail(1)),
    dropoffx_11=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(11).tail(1)),
    dropoffy_11=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(11).tail(1)),
    cost_11=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(11).tail(1)),
    collected_fare_type_11=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(11).tail(1)),
    customerID_12=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(12).tail(1)),
    startdatetime_12=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(12).tail(1)),
    pickupx_12=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(12).tail(1)),
    pickupy_12=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(12).tail(1)),
    enddatetime_12=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(12).tail(1)),
    dropoffx_12=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(12).tail(1)),
    dropoffy_12=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(12).tail(1)),
    cost_12=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(12).tail(1)),
    collected_fare_type_12=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(12).tail(1)),
    customerID_13=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(13).tail(1)),
    startdatetime_13=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(13).tail(1)),
    pickupx_13=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(13).tail(1)),
    pickupy_13=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(13).tail(1)),
    enddatetime_13=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(13).tail(1)),
    dropoffx_13=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(13).tail(1)),
    dropoffy_13=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(13).tail(1)),
    cost_13=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(13).tail(1)),
    collected_fare_type_13=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(13).tail(1)),
    customerID_14=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(14).tail(1)),
    startdatetime_14=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(14).tail(1)),
    pickupx_14=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(14).tail(1)),
    pickupy_14=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(14).tail(1)),
    enddatetime_14=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(14).tail(1)),
    dropoffx_14=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(14).tail(1)),
    dropoffy_14=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(14).tail(1)),
    cost_14=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(14).tail(1)),
    collected_fare_type_14=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(14).tail(1)),
    customerID_15=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(15).tail(1)),
    startdatetime_15=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(15).tail(1)),
    pickupx_15=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(15).tail(1)),
    pickupy_15=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(15).tail(1)),
    enddatetime_15=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(15).tail(1)),
    dropoffx_15=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(15).tail(1)),
    dropoffy_15=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(15).tail(1)),
    cost_15=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(15).tail(1)),
    collected_fare_type_15=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(15).tail(1)),
    customerID_16=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(16).tail(1)),
    startdatetime_16=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(16).tail(1)),
    pickupx_16=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(16).tail(1)),
    pickupy_16=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(16).tail(1)),
    enddatetime_16=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(16).tail(1)),
    dropoffx_16=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(16).tail(1)),
    dropoffy_16=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(16).tail(1)),
    cost_16=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(16).tail(1)),
    collected_fare_type_16=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(16).tail(1)),
    customerID_17=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(17).tail(1)),
    startdatetime_17=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(17).tail(1)),
    pickupx_17=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(17).tail(1)),
    pickupy_17=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(17).tail(1)),
    enddatetime_17=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(17).tail(1)),
    dropoffx_17=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(17).tail(1)),
    dropoffy_17=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(17).tail(1)),
    cost_17=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(17).tail(1)),
    collected_fare_type_17=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(17).tail(1)),
    customerID_18=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(18).tail(1)),
    startdatetime_18=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(18).tail(1)),
    pickupx_18=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(18).tail(1)),
    pickupy_18=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(18).tail(1)),
    enddatetime_18=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(18).tail(1)),
    dropoffx_18=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(18).tail(1)),
    dropoffy_18=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(18).tail(1)),
    cost_18=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(18).tail(1)),
    collected_fare_type_18=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(18).tail(1))
)
    
    
df_routes.to_csv("testing.csv")

             total_customers  customerID_1     startdatetime_1   pickupx_1  \
RunActualID                                                                  
2470597                    1         68809 2019-01-01 00:44:15 -105.193555   
2470981                    1         56045 2019-01-01 01:28:00 -105.057501   
2470985                    5         95414 2019-01-01 04:22:12 -105.022696   
2470991                    2         85450 2019-01-01 10:09:04 -105.071996   
2470993                    3         50497 2019-01-01 04:30:10 -104.994625   

             pickupy_1       enddatetime_1  dropoffx_1  dropoffy_1  cost_1  \
RunActualID                                                                  
2470597      39.723181 2019-01-01 01:35:22 -104.807529   39.640781     0.0   
2470981      39.812276 2019-01-01 02:22:00 -105.032882   39.638608     0.0   
2470985      39.708052 2019-01-01 04:40:18 -105.064721   39.711141     0.0   
2470991      39.744652 2019-01-01 10:58:17 -105.022603   39.762